In [24]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
import pickle
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [2]:
# Load dataset
dataset = pd.read_csv("cleaned_kidney_disease.csv")

In [3]:
dataset

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0.0,48.0,80.0,1.0200,1.0,0.000000,normal,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1.0,9.0,55.0,1.0200,4.0,0.000000,normal,normal,notpresent,notpresent,...,38,6000,5.2,no,no,no,good,no,no,ckd
2,2.0,62.0,80.0,1.0100,2.0,1.125356,normal,normal,notpresent,notpresent,...,31,7500,5.2,no,yes,no,poor,no,yes,ckd
3,3.0,48.0,70.0,1.0075,4.0,0.000000,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4.0,51.0,80.0,1.0100,2.0,0.000000,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395.0,55.0,80.0,1.0200,0.0,0.000000,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396.0,42.0,70.0,1.0250,0.0,0.000000,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397.0,12.0,80.0,1.0200,0.0,0.000000,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398.0,17.0,60.0,1.0250,0.0,0.000000,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [4]:
df = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [5]:
df

,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes,classification_ckd\t,classification_notckd
0,0.0,48.0,80.0,1.0200,1.0,0.000000,121.000000,36.0,1.2,137.528754,...,0,0,1,1,0,0,0,0,0,0
1,1.0,9.0,55.0,1.0200,4.0,0.000000,148.036517,18.0,0.8,137.528754,...,0,1,0,1,0,0,0,0,0,0
2,2.0,62.0,80.0,1.0100,2.0,1.125356,223.500000,53.0,1.8,137.528754,...,0,0,1,1,0,1,0,1,0,0
3,3.0,48.0,70.0,1.0075,4.0,0.000000,117.000000,56.0,3.8,126.000000,...,0,1,0,1,0,1,1,1,0,0
4,4.0,51.0,80.0,1.0100,2.0,0.000000,106.000000,26.0,1.4,137.528754,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395.0,55.0,80.0,1.0200,0.0,0.000000,140.000000,49.0,0.5,150.000000,...,0,1,0,1,0,0,0,0,0,1
396,396.0,42.0,70.0,1.0250,0.0,0.000000,75.000000,31.0,1.2,141.000000,...,0,1,0,1,0,0,0,0,0,1
397,397.0,12.0,80.0,1.0200,0.0,0.000000,100.000000,26.0,0.6,137.000000,...,0,1,0,1,0,0,0,0,0,1
398,398.0,17.0,60.0,1.0250,0.0,0.000000,114.000000,50.0,1.0,135.000000,...,0,1,0,1,0,0,0,0,0,1


In [6]:
df=df.drop('id', axis=1)

In [7]:
df

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes,classification_ckd\t,classification_notckd
0,48.0,80.0,1.0200,1.0,0.000000,121.000000,36.0,1.2,137.528754,4.627244,...,0,0,1,1,0,0,0,0,0,0
1,9.0,55.0,1.0200,4.0,0.000000,148.036517,18.0,0.8,137.528754,4.627244,...,0,1,0,1,0,0,0,0,0,0
2,62.0,80.0,1.0100,2.0,1.125356,223.500000,53.0,1.8,137.528754,4.627244,...,0,0,1,1,0,1,0,1,0,0
3,48.0,70.0,1.0075,4.0,0.000000,117.000000,56.0,3.8,126.000000,2.800000,...,0,1,0,1,0,1,1,1,0,0
4,51.0,80.0,1.0100,2.0,0.000000,106.000000,26.0,1.4,137.528754,4.627244,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.0200,0.0,0.000000,140.000000,49.0,0.5,150.000000,4.900000,...,0,1,0,1,0,0,0,0,0,1
396,42.0,70.0,1.0250,0.0,0.000000,75.000000,31.0,1.2,141.000000,3.500000,...,0,1,0,1,0,0,0,0,0,1
397,12.0,80.0,1.0200,0.0,0.000000,100.000000,26.0,0.6,137.000000,4.400000,...,0,1,0,1,0,0,0,0,0,1
398,17.0,60.0,1.0250,0.0,0.000000,114.000000,50.0,1.0,135.000000,4.900000,...,0,1,0,1,0,0,0,0,0,1


In [8]:
# Define input (independent variables) and output (dependent variable)
X = df.drop('classification_notckd', axis=1)
y = df['classification_notckd']

In [9]:
X

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,dm_\tyes,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes,classification_ckd\t
0,48.0,80.0,1.0200,1.0,0.000000,121.000000,36.0,1.2,137.528754,4.627244,...,0,0,0,1,1,0,0,0,0,0
1,9.0,55.0,1.0200,4.0,0.000000,148.036517,18.0,0.8,137.528754,4.627244,...,0,0,1,0,1,0,0,0,0,0
2,62.0,80.0,1.0100,2.0,1.125356,223.500000,53.0,1.8,137.528754,4.627244,...,0,0,0,1,1,0,1,0,1,0
3,48.0,70.0,1.0075,4.0,0.000000,117.000000,56.0,3.8,126.000000,2.800000,...,0,0,1,0,1,0,1,1,1,0
4,51.0,80.0,1.0100,2.0,0.000000,106.000000,26.0,1.4,137.528754,4.627244,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.0200,0.0,0.000000,140.000000,49.0,0.5,150.000000,4.900000,...,0,0,1,0,1,0,0,0,0,0
396,42.0,70.0,1.0250,0.0,0.000000,75.000000,31.0,1.2,141.000000,3.500000,...,0,0,1,0,1,0,0,0,0,0
397,12.0,80.0,1.0200,0.0,0.000000,100.000000,26.0,0.6,137.000000,4.400000,...,0,0,1,0,1,0,0,0,0,0
398,17.0,60.0,1.0250,0.0,0.000000,114.000000,50.0,1.0,135.000000,4.900000,...,0,0,1,0,1,0,0,0,0,0


In [10]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    1
399    1
Name: classification_notckd, Length: 400, dtype: int32

In [11]:
# Preprocess features (scaling)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

In [12]:
scaler

StandardScaler()

In [13]:
X_scaled_df

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,dm_\tyes,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes,classification_ckd\t
0,-0.213326,0.391532,0.483491,-0.013338,-0.557228,-0.347722,-0.496647,-0.567079,-0.111144,0.302593,...,-0.070888,-0.050063,-1.362770,1.408927,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888
1,-2.544835,-1.894959,0.483491,2.347516,-0.557228,0.240394,-1.092210,-0.782263,-0.111144,0.302593,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888
2,0.623626,0.391532,-1.413803,0.773613,2.258765,1.881924,0.065829,-0.244303,-0.111144,0.302593,...,-0.070888,-0.050063,-1.362770,1.408927,0.314485,-0.304789,1.969276,-0.484322,2.380476,-0.070888
3,-0.213326,-0.523064,-1.888126,2.347516,-0.557228,-0.434732,0.165090,0.831616,-2.296809,-2.540136,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,1.969276,2.064742,2.380476,-0.070888
4,-0.033979,0.391532,-1.413803,0.773613,-0.557228,-0.674012,-0.827515,-0.459487,-0.111144,0.302593,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.205150,0.391532,0.483491,-0.800290,-0.557228,0.065578,-0.066518,-0.943651,2.253203,0.726933,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888
396,-0.572019,-0.523064,1.432138,-0.800290,-0.557228,-1.348344,-0.662081,-0.567079,0.546949,-1.451113,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888
397,-2.365488,0.391532,0.483491,-0.800290,-0.557228,-0.804527,-0.827515,-0.889855,-0.211387,-0.050941,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888
398,-2.066576,-1.437660,1.432138,-0.800290,-0.557228,-0.499990,-0.033431,-0.674671,-0.590554,0.726933,...,-0.070888,-0.050063,0.733799,-0.709760,0.314485,-0.304789,-0.507801,-0.484322,-0.420084,-0.070888


In [14]:
# Feature Selection using SelectKBest with f_classif
skb = SelectKBest(f_classif, k=5)  # Select top 5 features
X_selected = skb.fit_transform(X_scaled, y)
selected_features = X.columns[skb.get_support()].tolist()

In [15]:
selected_features

['sg', 'al', 'hemo', 'htn_yes', 'dm_no']

In [16]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [17]:
# Initialize and train Decision Tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
print(f"Decision Tree Selected Features: {selected_features}")
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred_dt):.2f}")
print(classification_report(y_test, y_pred_dt))

Decision Tree Selected Features: ['sg', 'al', 'hemo', 'htn_yes', 'dm_no']
Decision Tree Accuracy: 0.99
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        52
           1       0.97      1.00      0.98        28

    accuracy                           0.99        80
   macro avg       0.98      0.99      0.99        80
weighted avg       0.99      0.99      0.99        80



In [18]:
# Initialize and train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f"SVM Selected Features: {selected_features}")
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")
print(classification_report(y_test, y_pred_svm))

SVM Selected Features: ['sg', 'al', 'hemo', 'htn_yes', 'dm_no']
SVM Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        52
           1       1.00      0.96      0.98        28

    accuracy                           0.99        80
   macro avg       0.99      0.98      0.99        80
weighted avg       0.99      0.99      0.99        80



In [19]:
# Initialize and train Logistic Regression model
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)
print(f"Logistic Regression Selected Features: {selected_features}")
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_log):.2f}")
print(classification_report(y_test, y_pred_log))

Logistic Regression Selected Features: ['sg', 'al', 'hemo', 'htn_yes', 'dm_no']
Logistic Regression Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        52
           1       1.00      0.96      0.98        28

    accuracy                           0.99        80
   macro avg       0.99      0.98      0.99        80
weighted avg       0.99      0.99      0.99        80



In [20]:
# Initialize and train XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(f"XGBoost Selected Features: {selected_features}")
print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb):.2f}")
print(classification_report(y_test, y_pred_xgb))

XGBoost Selected Features: ['sg', 'al', 'hemo', 'htn_yes', 'dm_no']
XGBoost Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        52
           1       1.00      0.96      0.98        28

    accuracy                           0.99        80
   macro avg       0.99      0.98      0.99        80
weighted avg       0.99      0.99      0.99        80



C:\Users\saran\anaconda3\envs\aimachine\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:18:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [21]:
# Initialize and train AdaBoost model
ada_model = AdaBoostClassifier()
ada_model.fit(X_train, y_train)
y_pred_ada = ada_model.predict(X_test)
print(f"AdaBoost Selected Features: {selected_features}")
print(f"AdaBoost Accuracy: {accuracy_score(y_test, y_pred_ada):.2f}")
print(classification_report(y_test, y_pred_ada))


AdaBoost Selected Features: ['sg', 'al', 'hemo', 'htn_yes', 'dm_no']
AdaBoost Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        52
           1       1.00      1.00      1.00        28

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



C:\Users\saran\anaconda3\envs\aimachine\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [25]:
# Save the scaler for future use during deployment
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [28]:
# Step 5: Save the trained model for deployment
joblib.dump(ada_model, 'ada_model.pkl')

['ada_model.pkl']

In [29]:
loaded_model = joblib.load('ada_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')


In [33]:
# New input data for prediction (replace with actual new data for deployment)
new_data = pd.DataFrame({
    'sg': [12],  
    'al': [12],  
    'hemo': [1],
    'htn_yes': [1],
    'dm_no': [1],})

In [34]:
new_data

,sg,al,hemo,htn_yes,dm_no
0,12,12,1,1,1


In [35]:
# Apply the saved scaler to transform the new data
new_data_scaled = loaded_scaler.transform(new_data)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- age
- ane_yes
- appet_poor
- ba_present
- bgr
- ...
